In [1]:
import sys
sys.path.append("~/Skrivbord/Oskar ET/ET/run_et/pytobii-master/pytobii")


import numpy as np
import time
import ipyleaflet 
import geocoder
from ipyleaflet import Map, FullScreenControl, Icon, Marker, AwesomeIcon, WidgetControl
import geopy
from geopy import distance
#from getdata import createdata
import ipywidgets as widgets
import threading
import pyautogui


from main_otherfreq import Eyetracker


class Karta():
    def __init__(self, place,basezoom,nrofenemies,nrofneutrals):
        self.location = geocoder.osm(place)
        self.homelatlng=[self.location.lat, self.location.lng] #get latitude and longitude
        self.basezoom=basezoom 
        self.nrofenemies=nrofenemies
        self.nrofneutrals=nrofneutrals
        self.m=Map(zoom=basezoom,dragging=False,scroll_wheel_zoom=True, center=self.homelatlng) #initialize map
        self.objects= np.zeros((2,1))
        self.switchon=False
        #-------Button for selection -----------
        #self.button = widgets.Button(description="Selection Off")
        #self.button.on_click(self.on_button_clicked)
        #self.button.style.button_color = 'red'
        #-------Button for removing select icon-----------
        self.button1 = widgets.Button(description="Remove marker")
        self.button1.on_click(self.on_button_clicked1)
        #self.button1.style.button_color = ''
        self.eyeicon = AwesomeIcon( name='eye',  marker_color='black', icon_color='red', spin=False)
        
        
        self.selecticon = AwesomeIcon( name='check',  marker_color='black', icon_color='yellow', spin=False)
        self.selectmarker=Marker(icon=self.selecticon ,draggable=False,keyboard=True,title="Selected", location=[])
        self.markerout=False
      
        
        

        
        self.zoomlvl=basezoom #Keeps track of current zoomlevel, for distance measuring
        #self.selectedicon = AwesomeIcon( name='eye',  marker_color='black', icon_color='red', spin=True) 
        #self.selectedmarker=Marker(icon=icon ,draggable=False,keyboard=True)
        #Todo: Draw cross on selected object, placed here in order to be altered from within whole class
        
        
    def iniate_map_with_home(self):
        self.m.add_control(FullScreenControl()) # Make fullscreen available 
        home_ship = AwesomeIcon( name='ship',  marker_color='blue', icon_color='black', spin=False) #Icon for home
        home= Marker(icon=home_ship ,draggable=False, location=(self.homelatlng)) #marker for home
        self.objects[0][0]=self.homelatlng[0] #Adding home coordinates to object list
        self.objects[1][0]=self.homelatlng[1]
        self.m.add_layer(home) #add home marker
        display(self.m)
        
        
        
    def add_random_enemies(self):
        #low and high values set to fix a bounding box around the nordic countries
            
            latituderange=np.random.uniform(low=54, high=70, size=(1,self.nrofenemies)) #Init coordinates
            longituderange=np.random.uniform(low=2, high=31, size=(1,self.nrofenemies))    
            enemy_coordinates=[latituderange[0],longituderange[0]]
            self.objects=np.concatenate([self.objects,enemy_coordinates], axis=1 )
            icon = AwesomeIcon( name='fighter-jet',  marker_color='red', icon_color='black', spin=False)
            for i in range(nrofenemies):
                enemy=Marker(icon=icon ,draggable=False,rise_on_hover=True,location=([latituderange[0][i],longituderange[0][i]]))
                self.m.add_layer(enemy)
                  
    def add_random_neutrals(self):
        #low and high values set to fix a bounding box around the nordic countries
            latituderange=np.random.uniform(low=54, high=70, size=(1,self.nrofneutrals))
            longituderange=np.random.uniform(low=2, high=31, size=(1,self.nrofneutrals))
            neutral_coordinates=[latituderange[0],longituderange[0]]
            self.objects=np.concatenate([self.objects,neutral_coordinates], axis=1 )
            icon = AwesomeIcon( name='plane',  marker_color='green', icon_color='black', spin=False)
            for i in range(nrofneutrals):
                neutral=Marker(icon=icon ,draggable=False, location=([latituderange[0][i],longituderange[0][i]]))
                self.m.add_layer(neutral)
            
           
                
    
    def add_specific_enemy(self):
        enemy=[[59.369172,59.254290],[18.334235,18.651818]]
        self.objects=np.concatenate([self.objects,enemy], axis=1 )
        icon = AwesomeIcon( name='fighter-jet',  marker_color='red', icon_color='black', spin=False)
        for i in range(2):
            specificenemy=Marker(icon=icon ,draggable=False, location=[enemy[0][i],enemy[1][i]])
            self.m.add_layer(specificenemy)
        
        
    def shortest_distance(self, point):
        threshold=200 #Threshold set to 200km feasible for the given zoom level. Should be set in another coordinate system later on
        distances=[geopy.distance.distance( (self.objects[0][i],self.objects[1][i]),(point[0],point[1]) ).km for i in range(np.size(self.objects,1))]
        # If both two least distances under a threshold go into zoom. Based on lat,long or other? :o
        #minimum, nextminimum = np.partition(distances, 1)[0:2]
       
        #if minimum<=threshold && nextminimum <=threshold:
        #   print("Zoom needed!")
        #    return
        
        if min(distances)<=threshold:
            if self.markerout==True:
                self.m.remove_layer(self.selectmarker)
                self.markerout=False
            #adda något med select marker här för GUIv4 för att kunna 
            #ha kvar marker för att kunna bestämma när ny marker ska sättas
            #möjligtvis sätta på annan knapp för att simulera eller ska den vara under den andra knappen
            #sätt under samma som on button clicked men dätt in en self variabel som oxå ska va ibockad under 
            #if satsen ovan
            closest_latlng=[self.objects[0][distances.index(min(distances))],self.objects[1][distances.index(min(distances))] ]
            
            self.selectmarker.location=closest_latlng
            #upp med selectmarker under init och ändra location via self.selectmarker.location=closest_latlng
            self.m.add_layer(self.selectmarker)
            self.markerout=True
            
            
            
            
        # Om 2 (heuristiskt vald, så fort två är för nära behövs zoosm) har distance inom en threshold från punkten krävs 
        # zoom,  sedan måste punkter göras om 
        #Threshold måste ha med zoom level att göra för en koordinat i sthlm har samma distance 
        # lat/lng baserat oavsett zoomlevel
        
    #def on_button_clicked(self,*args):
    #    
    #     if self.count==0:
    #        self.button.style.button_color = 'green'
    #        self.button.description=' Selection On'
    #        
    #        self.switchon=True
    #        self.count=1
    #        return
    #    if self.count==1:
    #        self.button.style.button_color = 'red'
    #        self.button.description='Selection Off'
    #        self.switchon=False
    #        self.count=0
    #        return
        
    def on_button_clicked1(self,*args):
        self.markerout=False
        self.m.remove_layer(self.selectmarker)
        
 
    def check_zoom(self):
        #self.m.zoom=1 Såhär ändras zoom levels ändra
        # variabeln self.zoomlvl
        pass
        #Todo: algorithm to check if zoom is needed.
        
    
    def draw_ET_DATA(self,**kwargs):
        
        
       
        if kwargs.get('type')=='preclick':#kwargs.get('type') == 'click' or 
            locationclick=kwargs.get('coordinates')
            self.shortest_distance(locationclick)
            #self.m.add_layer(Marker(icon=self.eyeicon,draggable=False,keyboard=True, location=locationclick))
            
        
         
        #f self.switchon==True: #will mark closest
            
       
        #self.m.remove_layer(eye)
          

        
        
        
        
    




if __name__ == "__main__":
    
   
    #-------- Variables for map initiation -----------
    place='Stockholm' #Map center/ Home object location, in our case STHLM
    basezoom=5 #First zoom appearence
    nrofneutrals=10 #Number of neutral objects on map
    nrofenemies=3 #Number of enemies displayed on map
    
    #-------- Variables for data creation -----------
    
    latlow=45  
    lathigh=69
    longlow=-13
    longhigh=46
    nrofdatapoints=1000
    #datacreator=createdata(latlow,lathigh,longlow,longhigh,nrofdatapoints)
    #data=datacreator.generate_random() #random data
    #data=datacreator.testzoom()
    
    
    #--------------- Map initialization -----------
    map_sthlm=Karta(place,basezoom,nrofenemies,nrofneutrals)
    map_sthlm.iniate_map_with_home()
    map_sthlm.add_random_enemies()
    map_sthlm.add_random_neutrals()
    map_sthlm.add_specific_enemy() #Generate two enemies in the sthlm archipelago
    
    

    #------------------Draw data on map----------------------------
    
    
    
   

    #tracking_control = WidgetControl(widget=map_sthlm.button, position='bottomright')
    #map_sthlm.m.add_control(tracking_control)
    remove_select = WidgetControl(widget=map_sthlm.button1, position='bottomleft')
    map_sthlm.m.add_control(remove_select)
    
    
    map_sthlm.m.on_interaction(map_sthlm.draw_ET_DATA)
    
    
    
    
    #start Eyetracker 
    #et=Eyetracker()
    #et.start_eyetracker()
    #tread.start()
    #tread = threading.Thread(target=et.start_eyetracker())
    #tread.start()
    
    
    
    
    
    
    
    #---------------------------------TODO------------------------------------------
    
    # Create eye tracking data
    # * Add fixation algorithm based on data in draw_ET_data (for now print random data).
    #2) IF fixation, see if fixation is located withing circle/ select nearest (two versions) call from draw_ET_DATA.
    #3) FIX activation of data reading through button click, dont know where to place atm.
    #4) Relativ mapping så att dsistanser beroende på zoom kan tas ut
    


    






Map(center=[59.3251172, 18.0710935], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…